# Project 3: Decision Trees & Random Forests
## This Project gives extra points for the final grade

## **Due 29.01.2026 at 4 PM**

## Overview

### **Submit your project solution as a group of 2-4 people.**

### Tasks

1. **Implement Decision Tree** (2.5 Points)
- Implement a decision tree classifier from scratch using the ID3 algorithm.
- You are free to add additional functionalities to improve the performance of your decision tree.

2. **Implement Random Forest** (2 Points)
- Implement a random forest classifier from scratch using your decision tree implementation.
- You are free to choose the hyperparameters of your random forest implementation.
- You are free to add any additional functionalities to improve the performance of your random forest.

3. **Testing & Evaluation** (0.5 Points)
- Train and tune hyperparameters of your decision tree and random forest implementations using the training set (you can use sklearn for hyperparameter tuning).
- You **must reach at least 95.5% accuracy** on the testing set with your final model.
- The best performing model will be awarded a prize in the class.

### **NOTE**: 
#### You are not allowed to use sklearn, pytorch or similar Deep Learning frameworks for the algorithm implementations. For data splitting, scoring and hyperparameter tuning sklearn is allowed. Numpy is allowed everywhere.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

In [2]:
# Do not change this part
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target) # 0 = malignant, 1 = benign
random_state = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random_state, stratify=y
)

In [3]:
# Implement a decision tree classifier from scratch using the ID3 algorithm.
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        # feature index and threshold used for splitting
        self.feature = feature
        self.threshold = threshold
        # left and right child nodes
        self.left = left
        self.right = right
        self.value = value

    def is_leaf(self):
        # check if this node is a leaf
        return self.value is not None

class DecisionTreeID3:
    def __init__(self, max_depth=10, min_samples_split=2, n_features=None):
        # stopping conditions and feature subsampling
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        X = X.values if isinstance(X, pd.DataFrame) else X
        y = y.values if isinstance(y, pd.Series) else y
        self.n_features = X.shape[1] if self.n_features is None else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples = X.shape[0]
        n_labels = len(np.unique(y))

        # stopping criteria
        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            return Node(value=self._most_common_label(y))

        # randomly select features (for random forest compatibility)
        feat_idxs = np.random.choice(X.shape[1], self.n_features, replace=False)
        best_feat, best_thresh, best_gain = self._best_split(X, y, feat_idxs)

        # stop splitting if information gain is too small
        if best_gain <= 1e-7:
            return Node(value=self._most_common_label(y))

        # split data and grow subtrees
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self._grow_tree(X[left_idxs], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs], y[right_idxs], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        # try all selected features and possible thresholds
        for feat in feat_idxs:
            X_col = X[:, feat]
            values = np.unique(X_col)
            thresholds = (values[:-1] + values[1:]) / 2

            for thr in thresholds:
                gain = self._information_gain(y, X_col, thr)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat
                    split_thresh = thr

        return split_idx, split_thresh, best_gain

    def _information_gain(self, y, X_col, threshold):
        # entropy before split
        parent_entropy = self._entropy(y)
        left, right = self._split(X_col, threshold)

        # invalid split
        if len(left) == 0 or len(right) == 0:
            return 0

        # weighted entropy after split
        n = len(y)
        e_left = self._entropy(y[left])
        e_right = self._entropy(y[right])
        child_entropy = (len(left)/n) * e_left + (len(right)/n) * e_right

        return parent_entropy - child_entropy

    def _entropy(self, y):
        # compute entropy of labels
        hist = np.bincount(y)
        probs = hist / len(y)
        probs = probs[probs > 0]
        return -np.sum(probs * np.log2(probs))

    def _split(self, X_col, threshold):
        # split indices based on threshold
        left = np.where(X_col <= threshold)[0]
        right = np.where(X_col > threshold)[0]
        return left, right

    def _most_common_label(self, y):
        # return majority class
        return np.bincount(y).argmax()

    def predict(self, X):
        # predict class for each sample
        X = X.values if isinstance(X, pd.DataFrame) else X
        return np.array([self._traverse(x, self.root) for x in X])

    def _traverse(self, x, node):
         # traverse tree until leaf is reached
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse(x, node.left)
        return self._traverse(x, node.right)

In [4]:
# Implement a random forest classifier from scratch using decision tree implementation.
class RandomForestClassifier:
    def __init__(self, n_trees=50, max_depth=10, min_samples_split=2, n_features=None):
        # forest hyperparameters
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        np.random.seed(42)
        self.trees = []

        for _ in range(self.n_trees):
            # create and train each tree on a bootstrap sample using decision tree implemented above
            tree = DecisionTreeID3(
                max_depth=self.max_depth,
                min_samples_split=self.min_samples_split,
                n_features=self.n_features
            )

            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample = X.iloc[idxs]
            y_sample = y.iloc[idxs]

            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        # collect predictions from all trees
        preds = np.array([tree.predict(X) for tree in self.trees])
        preds = preds.T
        return np.array([np.bincount(row).argmax() for row in preds])

In [6]:
# Train and tune hyperparameters of decision tree and random forest implementations using the training set.

# Hyperparameter Search Space
n_trees_options = [20, 50, 100]
max_depth_options = [8, 9, 10]
n_features_options = [int(np.sqrt(X.shape[1])), 10]

# Further splitting training set into train and validation
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

best_acc = 0
best_params = None
best_model = None

print("Starting Hyperparameter Tuning...")

for n_trees in n_trees_options:
    for depth in max_depth_options:
        for n_feats in n_features_options:
            rf = RandomForestClassifier(
                n_trees=n_trees,
                max_depth=depth,
                n_features=n_feats
            )
            rf.fit(X_tr, y_tr)
            preds = rf.predict(X_val)
            acc = accuracy_score(y_val, preds)

            print(f"Tested: n_trees={n_trees}, depth={depth}, feats={n_feats} | Val Accuracy: {acc*100:.2f}%")

            if acc > best_acc:
                best_acc = acc
                best_params = (n_trees, depth, n_feats)
                best_model = rf
                
print("-" * 40)
print(f"Best validation accuracy: {best_acc*100:.2f}%")
print(f"Best parameters: {best_params}")

Starting Hyperparameter Tuning...
Tested: n_trees=20, depth=8, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=20, depth=8, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=20, depth=9, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=20, depth=9, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=20, depth=10, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=20, depth=10, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=50, depth=8, feats=5 | Val Accuracy: 94.51%
Tested: n_trees=50, depth=8, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=50, depth=9, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=50, depth=9, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=50, depth=10, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=50, depth=10, feats=10 | Val Accuracy: 94.51%
Tested: n_trees=100, depth=8, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=100, depth=8, feats=10 | Val Accuracy: 95.60%
Tested: n_trees=100, depth=9, feats=5 | Val Accuracy: 95.60%
Tested: n_trees=100, depth=9, feats=10 | Val Accurac

In [7]:
# Final evaluation on the test set
best_model.fit(X_train, y_train)
test_preds = best_model.predict(X_test)
test_acc = accuracy_score(y_test, test_preds)

print("Final Test Accuracy:", test_acc)

Final Test Accuracy: 0.9736842105263158
